# CMIP 6 catalog

The CMIP6 data catalog is hosted as a csv file in google cloud. We can read from this catalog, filter the datasets we want to work with, and then only load the data we need. This is a great way to work with large datasets without having to download everything.

In [1]:
import fsspec
import pandas as pd
import xarray as xr

In [2]:
cat = pd.read_csv(
    "https://storage.googleapis.com/cmip6/cmip6-zarr-consolidated-stores.csv"
)
cat.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,ps,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
1,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rsds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
2,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlus,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
3,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,rlds,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706
4,HighResMIP,CMCC,CMCC-CM2-HR4,highresSST-present,r1i1p1f1,Amon,psl,gn,gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...,NaN,20170706


Using `pandas` methods, we can filter and select the variables of interest. Then, we can use `xarray` to load the data and work with it. More information about the CMIP6 catalog can be found in the excel file attached.

In [3]:
data_query = cat.query(
    "activity_id == 'CMIP' & table_id == 'Amon' & variable_id == 'tas' & experiment_id == 'historical' & source_id == 'GFDL-CM4'"
)
data_query

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
8074,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Amon,tas,gr1,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701


In [4]:
xrdata = xr.open_zarr(fsspec.get_mapper(data_query.zstore.iloc[0]), consolidated=True)
xrdata

<xarray.Dataset> Size: 411MB
Dimensions:    (bnds: 2, lat: 180, lon: 288, time: 1980)
Coordinates:
  * bnds       (bnds) float64 16B 1.0 2.0
    height     float64 8B ...
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lon        (lon) float64 2kB 0.625 1.875 3.125 4.375 ... 356.9 358.1 359.4
    lon_bnds   (lon, bnds) float64 5kB dask.array<chunksize=(288, 2), meta=np.ndarray>
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Data variables:
    tas        (time, lat, lon) float32 411MB dask.array<chunksize=(600, 180, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36500.0
    comment:                <null ref>
    ...                     ...
    variable_id:            tas
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-08-07;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/e4193a02-6405-49b6-8ad3-65def741a4dd...
    version_id:             v20180701

We can also query variables with vertical levels and save the output using `xarray` methods. Thanks to zarr, we should be able to access only the data we need without downloading the entire dataset.

In [5]:
data_query = cat.query(
    "activity_id == 'CMIP' & table_id == 'Omon' & variable_id == 'thetao' & experiment_id == 'historical' & source_id == 'GFDL-CM4' & grid_label == 'gr'"
)
data_query

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
9979,CMIP,NOAA-GFDL,GFDL-CM4,historical,r1i1p1f1,Omon,thetao,gr,gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-CM4/histo...,NaN,20180701


In [6]:
xrdata = xr.open_zarr(fsspec.get_mapper(data_query.zstore.iloc[0]), consolidated=True)
xrdata

<xarray.Dataset> Size: 18GB
Dimensions:    (lat: 180, bnds: 2, lev: 35, lon: 360, time: 1980)
Coordinates:
  * lat        (lat) float64 1kB -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
    lat_bnds   (lat, bnds) float64 3kB dask.array<chunksize=(180, 2), meta=np.ndarray>
  * lev        (lev) float64 280B 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lev_bnds   (lev, bnds) float64 560B dask.array<chunksize=(35, 2), meta=np.ndarray>
  * lon        (lon) float64 3kB 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lon_bnds   (lon, bnds) float64 6kB dask.array<chunksize=(360, 2), meta=np.ndarray>
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    thetao     (time, lev, lat, lon) float32 18GB dask.array<chunksize=(12, 35, 180, 360), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36500.0
    comment:                <null ref>
    ...                     ...
    variable_id:            thetao
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-08-03;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/7a1a3afa-77dc-4c45-8692-472086856d0c
    version_id:             v20180701

In [7]:
xrdata.nbytes / 1e9 # in GB

17.96262132

We can see that the size of this dataset is almost 18GB. Since we are interested in only a region, we can filter the data and only load the region of interest. This will save us a lot of time and storage space.

In [8]:
subset_region = xrdata.sel(lat=slice(-5, 5), lon=slice(190, 240))
subset_region

<xarray.Dataset> Size: 139MB
Dimensions:    (lat: 10, bnds: 2, lev: 35, lon: 50, time: 1980)
Coordinates:
  * lat        (lat) float64 80B -4.5 -3.5 -2.5 -1.5 -0.5 0.5 1.5 2.5 3.5 4.5
    lat_bnds   (lat, bnds) float64 160B dask.array<chunksize=(10, 2), meta=np.ndarray>
  * lev        (lev) float64 280B 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lev_bnds   (lev, bnds) float64 560B dask.array<chunksize=(35, 2), meta=np.ndarray>
  * lon        (lon) float64 400B 190.5 191.5 192.5 193.5 ... 237.5 238.5 239.5
    lon_bnds   (lon, bnds) float64 800B dask.array<chunksize=(50, 2), meta=np.ndarray>
  * time       (time) object 16kB 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
    time_bnds  (time, bnds) object 32kB dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    thetao     (time, lev, lat, lon) float32 139MB dask.array<chunksize=(12, 35, 10, 50), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.0 UGRID-1.0
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  36500.0
    comment:                <null ref>
    ...                     ...
    variable_id:            thetao
    variant_info:           N/A
    variant_label:          r1i1p1f1
    status:                 2019-08-03;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/7a1a3afa-77dc-4c45-8692-472086856d0c
    version_id:             v20180701

In [9]:
subset_region.nbytes / 1e9 # in GB

0.1386498

This still might take a while, but it is much better than downloading the entire dataset.

In [10]:
subset_region.to_netcdf("subset_region.nc") # Save the subset to a NetCDF file